<a href="https://colab.research.google.com/github/mysend12/study/blob/main/3%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 협업 필터링 추천 시스템

어떤 아이템에 대해 비슷한 취향을 가진 사람들은 다른 아이템 또한 비슷한 취향을 가질 것이다.  
협업 필터링(Collaborative Filtering: CF)

## 원리
취향이 비슷한 사람들의 집단 존재한다고 가정  
추천 대상의 유사 집단을 찾아서 그들이 좋아하는 아이템을 추천  

CF에서는 사용자간의 유사도를 찾는것이 핵심

## 유사도 지표

### 상관계수
- 가장 이해하기 쉬운 유사도
- -1 ~ 1 사이의 값  
-- 1인 경우 양의 상관관계
-- -1에 가까우면 음의 상관관계
- 0인 경우 상관관계가 없거나, U 형 상관관계 등 일반적이지 않은 상관관계

### 코사인 유사도
1. 협업 필터링에서 가장 널리 쓰이는 유사도
2. 각 아이템 => 하나의 차원, 사용자의 평가값 => 좌표값
3. 두 사용자의 평가값 유사 => theta는 작아지고, 코사인 값은 커짐
4. -1 ~ 1 사이의 값
5. 데이터 이진값(binary) => 타니모토 계수(tanimoto coefficient) 사용 권장


### 자카드 계수
1. 타니모토 계수의 변형 => 자카드 계수
2. 이진수 데이터 => 좋은 결과


## 기본 CF 알고리즘
1. 모든 사용자 간 평가의 유사도 계산
2. 추천 대상과 다른 사용자간 유사도 추출
3. 추천 대상이 평가하지 않은 아이템에 대한 예상 평가값 계산
- 평가값 = 다른 사용자 평가 * 다른 사용자 유사도
4. 아이템 중 예상 평가값이 가장 높은 N개 추천

In [14]:
# 사용자 u.user 파일을 DataFrame으로 읽기

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

##### 데이터 불러오기 및 필요한 함수 정의 #####
base_src = 'drive/MyDrive/RecoSys/Data'

# user 데이터

# user
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    u_user_src,
    sep='|',
    names=u_cols,
    encoding='latin-1'
)
users = users.set_index('user_id')

# movie
u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id', 'title', 'release data', 'video release data', 'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animation',
          'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
          'Thriller', 'War', 'Western'
          ]
movies = pd.read_csv(u_item_src, sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')

# rating
u_data_src = os.path.join(base_src, 'u.data')
u_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(
    u_data_src,
    sep='\t',
    names=u_cols,
    encoding='latin-1'
)

def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# score(RMSE) 계산
def score(model):
  # 테스트 데이터의 user_id와 movie_id 간 pair를 맞춰 튜플형 원소 리스트 데이터를 만든다.
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  # 모든 사용자-영화 짝에 대해서 주어진 예측 모델에 의해 예측값 계산 및 리스트형 데이터 생성
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
  # 실제 평점값
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

##### 데이터 셋 만들기 #####
x = ratings.copy()
y= ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)
ratings_matrix = x_train.pivot(index='user_id', columns = 'movie_id', values='rating')

##### 코사인 유사도 계산 #####
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=ratings_matrix.index, columns=ratings_matrix.index)

##### 주어진 영화의(movie_id) 가중평균 rating을 계산하는 함수 #####
def CF_simple(user_id, movie_id):
  ## 지정한 movie_id가 rating metrix의 column 안에 있는 경우
  if movie_id in ratings_matrix.columns:
    ## 주어진 사용자와 다른 사용자간의 유사성을 추출하여 복제
    sim_scores = user_similarity[user_id].copy()
    ## 주어진 영화의 다른 사용자들에 의한 평가
    movie_ratings = ratings_matrix[movie_id].copy()
    ## 주어진 영화에 대해서 평가를 하지 않은 사용자의 인덱스를 추출
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    ## 평가하지 않은 사용자들을 영화 평점에서 제외
    movie_ratings = movie_ratings.dropna()
    ## 평가하지 않은 사용자들을 유사도에서도 제외
    sim_scores = sim_scores.drop(none_rating_idx)
    ## 평가한 사람들을 가중평균하여 나누기
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  else:
    mean_rating = 3.0
  return mean_rating

##### 정확도 계산 #####
score(CF_simple)



1.0209780684067873